# Lasso & Ridge VotingRegressor 14634.86548

In [ ]:
from sklearn.linear_model import Ridge

ridge_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('ridge', Ridge(alpha = 9.5410, max_iter=10000))
])

set_config(display="diagram")
ridge_pipeline.fit(X_train, y_train)

#############################

from sklearn.linear_model import Lasso


lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lasso', Lasso(alpha = 0.0005, max_iter=10000))
])

set_config(display="diagram")
lasso_pipeline.fit(X_train, y_train)


#######################


from sklearn.ensemble import VotingRegressor
ensemble_regressor = VotingRegressor([
    ('ridge', ridge_pipeline),
    ('lasso', lasso_pipeline)
])

ensemble_regressor.fit(X_train, y_train)


#######################

test = pd.read_csv("../input/test.csv")
# Save the test IDs for submission
test_ids = test["Id"]

# Apply the same column-dropping steps to the test data
test = test.drop(columns=[col for col in drop_list if col in test.columns], errors='ignore')


test['Years_Since_Remod'] = test['YrSold'] - test['YearRemodAdd']
test['Age']               = test['YrSold'] - test['YearBuilt']
test['Newness']           = test['Age'] * test['Years_Since_Remod']
test['Total_Home_Quality'] = test['OverallQual'] + test['OverallCond']



# In Kaggle test data, the target is not provided so drop "Id" and use the rest as features.
X_test = test.drop("Id", axis=1)

# ------------------------------
# 3. Make Predictions & Create Submission File
# ------------------------------
loged_predictions = ridge_pipeline.predict(X_test)
predictions = np.expm1(loged_predictions)
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": predictions
})

submission.to_csv("../submission.csv", index=False)
print("Submission file saved as submission.csv")